In [1]:
import pandas as pd
#Load libraries
from sklearn.preprocessing import PowerTransformer
import numpy as np
from kmodes.kprototypes import KPrototypes
#Load libraries
from matplotlib import style
from lightgbm import LGBMClassifier
import shap
from sklearn.model_selection import cross_val_score
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#Load library
from sklearn.preprocessing import LabelEncoder
import os
import random

/apps/jupyterhub/2019-04-29/miniconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/apps/jupyterhub/2019-04-29/miniconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/apps/jupyterhub/2019-04-29/miniconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/apps/jupyterhub/2019-04-29/miniconda

PREPROCESSING df_sup_not_processed : Supervised clustering

In [2]:
#Import dataset for supervised clustering
df_sup_unprocess = pd.read_csv('/rds/general/user/md2620/home/asthma/Malo/Dataframes/Final/df_sup_not_processed.csv', low_memory=False)

In [3]:
#Check if there are variables necessiting removal
df_sup_unprocess.head(5)

,Unnamed: 0,Biomarker.C5a.pg.ml.serum,Biomarker.CD40L.pg.ml.serum,Biomarker.DPPIV.pg.ml.serum,Biomarker.Galectin.3.pg.ml.serum,Biomarker.IL.18.pg.ml.serum,Biomarker.IL.1alpha.pg.ml.serum,Biomarker.IL.6Ralpha.pg.ml.serum,Biomarker.LBP.pg.ml.serum,Biomarker.MCP.4.pg.ml.serum,...,Blood.Eosinophils_.,Omics.Proteomics.Serum,Questions.Screening.Asthma.Father,Questions.Screening.Hay.Fever.Mother,Questions.Screening.Inflammatory.Disease.Father,Hay.Fever.Diagnosed,Non.Allergic.Rhinitis.Diagnosed,Parental.Asthma,Sex,cohort
0,A_151,24,6593,107201,3112,225,32,7593,1350961,123,...,2.703382,SOTON_SERUM,uncertain,no,no,yes,yes,uncertain,male,Severe
1,A_591,23,505,119893,4018,192,29,9263,1357902,97,...,1.610000,SOMALOGIC_SERUM,no,no,no,no,no,no,female,Healthy
2,A_630,38,5126,107393,5584,86,36,8837,1202367,217,...,2.800000,SOMALOGIC_SERUM,no,yes,no,no,no,yes,female,Healthy
3,A_589,63,4175,79825,4546,258,25,11102,3416341,160,...,5.000000,SOTON_SERUM,no,no,no,no,no,no,male,Severe
4,A_065,56,6143,57804,9354,225,48,10146,2596176,243,...,4.110000,SOTON_SERUM,no,no,no,no,no,no,female,Severe_Smoker


In [4]:
#Set Unnamed: 0 as index of df_sup_unprocess
df_sup_unprocess = df_sup_unprocess.set_index('Unnamed: 0')

In [9]:
#Check variable types for specific columns using:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df_sup_unprocess.iloc[:,0:105].dtypes

Biomarker.C5a.pg.ml.serum                                               int64
Biomarker.CD40L.pg.ml.serum                                             int64
Biomarker.DPPIV.pg.ml.serum                                             int64
Biomarker.Galectin.3.pg.ml.serum                                        int64
Biomarker.IL.18.pg.ml.serum                                             int64
Biomarker.IL.1alpha.pg.ml.serum                                         int64
Biomarker.IL.6Ralpha.pg.ml.serum                                        int64
Biomarker.LBP.pg.ml.serum                                               int64
Biomarker.MCP.4.pg.ml.serum                                             int64
Biomarker.MMP.3.pg.ml.serum                                             int64
Biomarker.RAGE.pg.ml.serum                                              int64
Biomarker.SHBG.pg.ml.serum                                              int64
Biomarker.Serpin.E1.pg.ml.serum                                 

In [10]:
#Standardize using yeo-johnson transformation numerical variables
from sklearn.preprocessing import PowerTransformer
for c in df_sup_unprocess.iloc[:,0:105]:
    pt = PowerTransformer()
    df_sup_unprocess[c] =  pt.fit_transform(np.array(df_sup_unprocess[c]).reshape(-1, 1))

divide by zero encountered in log
divide by zero encountered in log
divide by zero encountered in log
divide by zero encountered in log
divide by zero encountered in log
divide by zero encountered in log


In [11]:
#Save dataset for supervised K-proto in R
os.chdir("/rds/general/user/md2620/home/asthma/Malo/Dataframes/Final")
df_sup_unprocess.to_csv("df_sup_num_processed.csv")

FINAL PROCESSING SUP_DF

In [28]:
#Import dataset to process cat variables after having run supervised K-prototypes (K= 2 to 15 in R)
#Needing to construct shap and run K-proto in Python + to run PCA
df_sup_partial_process = pd.read_csv('/rds/general/user/md2620/home/asthma/Malo/Dataframes/Final/sup_df.csv', low_memory=False)

In [29]:
df_sup_partial_process.head(5)

,Unnamed: 0,Biomarker.C5a.pg.ml.serum,Biomarker.CD40L.pg.ml.serum,Biomarker.DPPIV.pg.ml.serum,Biomarker.Galectin.3.pg.ml.serum,Biomarker.IL.18.pg.ml.serum,Biomarker.IL.1alpha.pg.ml.serum,Biomarker.IL.6Ralpha.pg.ml.serum,Biomarker.LBP.pg.ml.serum,Biomarker.MCP.4.pg.ml.serum,Biomarker.MMP.3.pg.ml.serum,Biomarker.RAGE.pg.ml.serum,Biomarker.SHBG.pg.ml.serum,Biomarker.Serpin.E1.pg.ml.serum,Biomarker.alpha1.microglobulin.pg.ml.serum,Biomarker.CCL17.pg.ml.MSD.BL.plasma,Biomarker.EOTAXIN.pg.ml.MSD.BL.plasma,Biomarker.Baseline.BI.Cytokines.Chemokines.IL6.pg.ml.MSD.BL.plasma,Biomarker.MCP1.pg.ml.MSD.BL.plasma,Biomarker.MIP1b.pg.ml.MSD.BL.plasma,Biomarker.TNFa.pg.ml.MSD.BL.plasma,Biomarker.Genentech.CCL18.pg.ml.IMPACT.BL.serum,Biomarker.Genentech.IL13.pg.ml.IMPACT.BL.serum,Biomarker.Genentech.Periostin.ng.ml.ELECSYS.BL.serum,Biomarker.Karolinska.hsCRP.hCRP.mg.L,Clinical.Atopy.Total.IgE.IU.ml,Clinical.NO.Standard.Flow.Rate,Clinical.Screening.Albumin.g.dL,Clinical.Screening.Alkaline.Phosphatase.U.L,Clinical.Screening.Alt.U.L.,Clinical.Screening.Ast.U.L,Clinical.Screening.Blood.Urea.Nitrogen.mg.dL,Clinical.Screening.Creatinine.umol.L,Clinical.Screening.Gamma.Gt.U.L,Clinical.Screening.Lymphocytes,Clinical.Screening.Monocytes,Clinical.Screening.Platelets.x10.3.uL,Clinical.Screening.Sodium.mmol.L,Clinical.Screening.Total.Bilirubin.umol.L,Clinical.Screening.Total.Protein.g.dL,Clinical.Screening.Wbcs.x10.3.uL,Clinical.Screening.lymphocytes.x10.3.uL,Clinical.FEF.25.75.Absolute.Change.L.sec,Clinical.FEV1.FVC.Ratio.Predicted.LLN,Clinical.FVC.Absolute.Change.L,Clinical.Baseline.PEF,Clinical.Screening.Diastolic.Blood.Pressure,Clinical.Screening.Heart.Rate,Clinical.Screening.Respiratory.Rate,Clinical.Screening.Systolic.Blood.Pressure,Exacerbation.Per.Year.,Omics.dehydroTXB2.ng.mmolC,Omics.dinor.11.B.PGF2a.ng.mmolC,Omics.dinor.8.isoPGF2a.ng.mmolC,Omics.dinor.TXB2.ng.mmolC,Omics.isoPGF2a.ng.mmolC,Omics.LTE4.ng.mmolC,Omics.PGF2a.ng.mmolC,Omics.tetranorPGDM.ng.mmolC,Omics.tetranorPGEM.ng.mmolC,Omics.Trimethyluric.acid,Omics.Dimethyluric.acid,Omics.Methyluric.acid,Omics.Methylxanthine,Omics.Acetylamino.6.formylamino.3.methyluracil,Omics.Acetylcarnitine,Omics.Carnitine,Omics.Galacturonic.acid,Omics.Glutamine,Omics.Guanine,Omics.Histidine,Omics.Kynurenic.acid,Omics.Maltose,Omics.Mannitol,Omics.Methionine,Omics.Methylthioadenosine,Omics.N.Acetylcarnosine,Omics.N.Acetylglutamic.acid,Omics.N.Acetylputrescine,Omics.N.Methyl.D.aspartic.acid,Omics.N.Methylhistamine,Omics.Pipecolic.acid,Omics.Propionylcarnitine,Omics.S.Adenosylhomocysteine,Omics.Saccharopine,Omics.Sarcosine,Omics.Serine,Omics.Serotonin,Omics.Sucrose,Omics.Uracil,Omics.Xylose,Questionnaires.ACQ.FEV1.Precentage,Questionnaires.AQLQ.Emotional.Total.Imputed,Questionnaires.ESS.Total.Imputed,Questionnaires.HADS.Anxiety.Total.Imputed,Questionnaires.HADS.Depression.Total.Imputed,Questionnaires.SNOT.Total.Imputed,Body.Mass.Index.kg.m2,Height.cm,Respiratory.History.Onset.OR.First.Diagnosis.Age.years,Clinical.Screening.Haematology.Not.Done,Age,Sputum.._Eosinophils,Blood.Neutrophils_.,Sputum._Neutrophils,Blood.Eosinophils_.,Omics.Proteomics.Serum,Questions.Screening.Asthma.Father,Questions.Screening.Hay.Fever.Mother,Questions.Screening.Inflammatory.Disease.Father,Hay.Fever.Diagnosed,Non.Allergic.Rhinitis.Diagnosed,Parental.Asthma,Sex,cohort,cluster_K_2,cluster_K_3,cluster_K_4,cluster_K_5,cluster_K_6,cluster_K_7,cluster_K_8,cluster_K_9,cluster_K_10,cluster_K_11,cluster_K_12,cluster_K_13,cluster_K_14,cluster_K_15,Healthy,MildModerate,Severe,Severe_Smoker,Severe_asthma
0,A_151,-0.724095,0.863080,0.252664,-1.766019,0.001351,-0.210162,-1.539914,-1.036255,-0.107821,-1.245209,2.965697,0.032570,-1.013383,0.186077,-2.172842,0.930240,-0.789349,-0.438213,-1.522596,0.982285,-1.020478,-0.578457,-0.315207,-1.159050,1.231261,-1.181340,0.552621,-0.040440,-0.192761,-0.079863,0.388968,0.187291,0.857627,0.152694,0.237446,-0.240586,0.137042,-0.024990,-0.908405,-0.382681,-0.604908,0.744726,1.405463,1.416039,0.495466,1.108255,0.4405

In [32]:
#Check variable types for specific columns using:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df_sup_partial_process.iloc[:,106:114].dtypes

Omics.Proteomics.Serum                             object
Questions.Screening.Asthma.Father                  object
Questions.Screening.Hay.Fever.Mother               object
Questions.Screening.Inflammatory.Disease.Father    object
Hay.Fever.Diagnosed                                object
Non.Allergic.Rhinitis.Diagnosed                    object
Parental.Asthma                                    object
Sex                                                object
dtype: object

In [33]:
#Convert range of variables 103-111 to str
df_sup_partial_process.iloc[:,106:114]=df_sup_partial_process.iloc[:,106:114].astype('str')

In [34]:
#Label encode categorical variables
from sklearn import preprocessing
for c in df_sup_partial_process.iloc[:,106:114]:
    le = preprocessing.LabelEncoder()
    df_sup_partial_process[c] = le.fit_transform(np.array(df_sup_partial_process[c]).reshape(-1, 1))

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [35]:
#Standardize using yeo-johnson transformation for all variables in df to be scaled identically
from sklearn.preprocessing import PowerTransformer
for c in df_sup_partial_process.iloc[:,106:114]:
    pt = PowerTransformer()
    df_sup_partial_process[c] =  pt.fit_transform(np.array(df_sup_partial_process[c]).reshape(-1, 1))

In [36]:
df_sup_partial_process.head(5)

,Unnamed: 0,Biomarker.C5a.pg.ml.serum,Biomarker.CD40L.pg.ml.serum,Biomarker.DPPIV.pg.ml.serum,Biomarker.Galectin.3.pg.ml.serum,Biomarker.IL.18.pg.ml.serum,Biomarker.IL.1alpha.pg.ml.serum,Biomarker.IL.6Ralpha.pg.ml.serum,Biomarker.LBP.pg.ml.serum,Biomarker.MCP.4.pg.ml.serum,Biomarker.MMP.3.pg.ml.serum,Biomarker.RAGE.pg.ml.serum,Biomarker.SHBG.pg.ml.serum,Biomarker.Serpin.E1.pg.ml.serum,Biomarker.alpha1.microglobulin.pg.ml.serum,Biomarker.CCL17.pg.ml.MSD.BL.plasma,Biomarker.EOTAXIN.pg.ml.MSD.BL.plasma,Biomarker.Baseline.BI.Cytokines.Chemokines.IL6.pg.ml.MSD.BL.plasma,Biomarker.MCP1.pg.ml.MSD.BL.plasma,Biomarker.MIP1b.pg.ml.MSD.BL.plasma,Biomarker.TNFa.pg.ml.MSD.BL.plasma,Biomarker.Genentech.CCL18.pg.ml.IMPACT.BL.serum,Biomarker.Genentech.IL13.pg.ml.IMPACT.BL.serum,Biomarker.Genentech.Periostin.ng.ml.ELECSYS.BL.serum,Biomarker.Karolinska.hsCRP.hCRP.mg.L,Clinical.Atopy.Total.IgE.IU.ml,Clinical.NO.Standard.Flow.Rate,Clinical.Screening.Albumin.g.dL,Clinical.Screening.Alkaline.Phosphatase.U.L,Clinical.Screening.Alt.U.L.,Clinical.Screening.Ast.U.L,Clinical.Screening.Blood.Urea.Nitrogen.mg.dL,Clinical.Screening.Creatinine.umol.L,Clinical.Screening.Gamma.Gt.U.L,Clinical.Screening.Lymphocytes,Clinical.Screening.Monocytes,Clinical.Screening.Platelets.x10.3.uL,Clinical.Screening.Sodium.mmol.L,Clinical.Screening.Total.Bilirubin.umol.L,Clinical.Screening.Total.Protein.g.dL,Clinical.Screening.Wbcs.x10.3.uL,Clinical.Screening.lymphocytes.x10.3.uL,Clinical.FEF.25.75.Absolute.Change.L.sec,Clinical.FEV1.FVC.Ratio.Predicted.LLN,Clinical.FVC.Absolute.Change.L,Clinical.Baseline.PEF,Clinical.Screening.Diastolic.Blood.Pressure,Clinical.Screening.Heart.Rate,Clinical.Screening.Respiratory.Rate,Clinical.Screening.Systolic.Blood.Pressure,Exacerbation.Per.Year.,Omics.dehydroTXB2.ng.mmolC,Omics.dinor.11.B.PGF2a.ng.mmolC,Omics.dinor.8.isoPGF2a.ng.mmolC,Omics.dinor.TXB2.ng.mmolC,Omics.isoPGF2a.ng.mmolC,Omics.LTE4.ng.mmolC,Omics.PGF2a.ng.mmolC,Omics.tetranorPGDM.ng.mmolC,Omics.tetranorPGEM.ng.mmolC,Omics.Trimethyluric.acid,Omics.Dimethyluric.acid,Omics.Methyluric.acid,Omics.Methylxanthine,Omics.Acetylamino.6.formylamino.3.methyluracil,Omics.Acetylcarnitine,Omics.Carnitine,Omics.Galacturonic.acid,Omics.Glutamine,Omics.Guanine,Omics.Histidine,Omics.Kynurenic.acid,Omics.Maltose,Omics.Mannitol,Omics.Methionine,Omics.Methylthioadenosine,Omics.N.Acetylcarnosine,Omics.N.Acetylglutamic.acid,Omics.N.Acetylputrescine,Omics.N.Methyl.D.aspartic.acid,Omics.N.Methylhistamine,Omics.Pipecolic.acid,Omics.Propionylcarnitine,Omics.S.Adenosylhomocysteine,Omics.Saccharopine,Omics.Sarcosine,Omics.Serine,Omics.Serotonin,Omics.Sucrose,Omics.Uracil,Omics.Xylose,Questionnaires.ACQ.FEV1.Precentage,Questionnaires.AQLQ.Emotional.Total.Imputed,Questionnaires.ESS.Total.Imputed,Questionnaires.HADS.Anxiety.Total.Imputed,Questionnaires.HADS.Depression.Total.Imputed,Questionnaires.SNOT.Total.Imputed,Body.Mass.Index.kg.m2,Height.cm,Respiratory.History.Onset.OR.First.Diagnosis.Age.years,Clinical.Screening.Haematology.Not.Done,Age,Sputum.._Eosinophils,Blood.Neutrophils_.,Sputum._Neutrophils,Blood.Eosinophils_.,Omics.Proteomics.Serum,Questions.Screening.Asthma.Father,Questions.Screening.Hay.Fever.Mother,Questions.Screening.Inflammatory.Disease.Father,Hay.Fever.Diagnosed,Non.Allergic.Rhinitis.Diagnosed,Parental.Asthma,Sex,cohort,cluster_K_2,cluster_K_3,cluster_K_4,cluster_K_5,cluster_K_6,cluster_K_7,cluster_K_8,cluster_K_9,cluster_K_10,cluster_K_11,cluster_K_12,cluster_K_13,cluster_K_14,cluster_K_15,Healthy,MildModerate,Severe,Severe_Smoker,Severe_asthma
0,A_151,-0.724095,0.863080,0.252664,-1.766019,0.001351,-0.210162,-1.539914,-1.036255,-0.107821,-1.245209,2.965697,0.032570,-1.013383,0.186077,-2.172842,0.930240,-0.789349,-0.438213,-1.522596,0.982285,-1.020478,-0.578457,-0.315207,-1.159050,1.231261,-1.181340,0.552621,-0.040440,-0.192761,-0.079863,0.388968,0.187291,0.857627,0.152694,0.237446,-0.240586,0.137042,-0.024990,-0.908405,-0.382681,-0.604908,0.744726,1.405463,1.416039,0.495466,1.108255,0.4405

In [37]:
#Save final supervised dataset with processing of cat variables
os.chdir("/rds/general/user/md2620/home/asthma/Malo/Dataframes/Final")
df_sup_partial_process.to_csv("sup_df_all_processed.csv")